In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

col_names = ['vid', 'lat', 'lng', 'type', 'country']
dataPOIs = pd.read_csv('../datasets_original/foursquare/POIs.txt', sep='\t', header=None, names=col_names)

In [2]:
vids = []
for index, row in dataPOIs.iterrows():
    if row['country'] == 'US':
        vids.append(row['vid'])

In [3]:
len(vids)

501900

In [4]:
df = pd.read_csv('../datasets_final/foursquare/dataset.csv')
df_final = df[df.vid.isin(vids)]
df_final['datetime'] = pd.to_datetime(df_final.datetime).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
groups = df_final.groupby('uid')
sizes = groups.size()
min_checkins = 100
users = sizes[sizes>=min_checkins].index
df_final2 = df_final[df_final['uid'].isin(users)]

groups2 = df_final2.groupby('vid')
sizes2 = groups2.size()
min_occurences = 50
places = sizes2[sizes2>=min_occurences].index
df_final3 = df_final2[df_final2['vid'].isin(places)]
df_results = df_final3.sort_values(['uid', 'datetime'], ascending = (True, False))
print(df_results.uid.nunique())
print(df_results.vid.nunique())

/tmp/ipykernel_2709099/802885252.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['datetime'] = pd.to_datetime(df_final.datetime).dt.strftime('%Y-%m-%dT%H:%M:%SZ')


11116
7183


In [5]:
vocabulary = list(df_results.vid.unique())
vid_indices = dict((c, i) for i, c in enumerate(vocabulary))
indices_vid = dict((i, c) for i, c in enumerate(vocabulary))
newList = [vid_indices[k] for k in df_results.vid if k in vid_indices]
df_results.vid = newList
df_results = df_results.astype({'vid':'int'})
df_results.to_csv(r'../datasets_final/foursquare_US/dataset.txt', sep="\t", mode='a', header=False, index=False)
df_results

,uid,datetime,lat,lng,vid
1872,1,2013-03-12T20:34:32Z,40.694416,-73.675675,0
1870,1,2013-03-12T20:33:08Z,40.694334,-73.678330,1
1869,1,2013-03-12T20:32:54Z,40.694475,-73.678415,2
1868,1,2013-03-12T18:44:14Z,40.694298,-73.678324,3
1867,1,2013-03-12T18:43:40Z,40.694475,-73.678415,2
...,...,...,...,...,...
25294761,187809,2013-04-17T20:56:53Z,27.946777,-82.526529,2839
25294741,187809,2013-03-06T22:49:22Z,28.069187,-82.576627,1840
25294691,187809,2012-12-28T03:34:30Z,28.033103,-82.542579,1842
25295146,187855,2013-03-30T03:36:52Z,39.778464,-86.159635,2355
